In [178]:
import pandas as pd
import numpy as np

datapath = '../data/'
hetrec = datapath + 'hetrec2011-movielens-2k-v2/'

user_item_matrix = pd.read_csv(datapath + 'ml-latest-small/ratings.csv', usecols=[0, 1, 2]).rename(columns={'movieId':'movieID'})
movies = pd.read_csv(datapath + 'ml-latest-small/movies.csv', usecols=[0, 1])

movie_tags = pd.read_csv(hetrec + 'movie_tags.dat', sep='\t')
# movie_tags['list'] = movie_tags.apply(lambda row: [row['tagID']] * row['tagWeight'], axis=1)
movie_tags['list'] = movie_tags.apply(lambda row: [row['tagID']], axis=1)
new_tags = movie_tags.groupby('movieID')['list'].sum()
user_item_matrix['tags'] = user_item_matrix.apply(lambda row: new_tags.get(row['movieID']), axis=1)
user_item_matrix = user_item_matrix.dropna()
to_pad = max(user_item_matrix['tags'].apply(lambda value: len(value)))
user_item_matrix['tags'] = user_item_matrix['tags'].apply(lambda value: np.array([0]*(to_pad-len(value))+value))

movie_genres = pd.read_csv(hetrec + 'movie_genres.dat', sep='\t')
genre_dict = {v: k+1 for k, v in enumerate(movie_genres['genre'].unique())}
movie_genres['genre'] = movie_genres['genre'].replace(genre_dict)
movie_genres['list'] = movie_genres.apply(lambda row: [row['genre']], axis=1)
new_genres = movie_genres.groupby('movieID')['list'].sum()
user_item_matrix['genres'] = user_item_matrix.apply(lambda row: new_genres.get(row['movieID']), axis=1)
to_pad = max(user_item_matrix['genres'].apply(lambda value: len(value)))
user_item_matrix['genres'] = user_item_matrix['genres'].apply(lambda value: np.array([0]*(to_pad-len(value))+value))

countries = pd.read_csv(hetrec + 'movie_countries.dat', sep='\t')
user_item_matrix = pd.merge(user_item_matrix, countries, on='movieID').sort_values(by=['userId', 'movieID'])
country_dict = {v: k+1 for k, v in enumerate(countries['country'].unique())}
user_item_matrix["country"] = user_item_matrix["country"].map(country_dict)

# remapping movieIDs so that torch stop yelling at me
new_dict = {a: b+1 for b, a in enumerate(sorted(user_item_matrix['movieID'].unique()))}
user_item_matrix['movieID'] = user_item_matrix['movieID'].map(new_dict)
user_item_matrix = user_item_matrix.dropna()
user_item_matrix.head()

,userId,movieID,rating,tags,genres
0,1,1,4.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 2, 3, 4, 5, 0, 0, 0]"
1,1,3,4.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4, 6, 0, 0, 0, 0, 0, 0]"
2,1,5,4.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[8, 9, 10, 0, 0, 0, 0, 0]"
3,1,41,5.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[9, 11, 12, 10, 0, 0, 0, 0]"
4,1,44,5.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[9, 12, 10, 0, 0, 0, 0, 0]"


In [181]:
user_item_matrix['genres'].values

array([array([1, 2, 3, 4, 5, 0, 0, 0]), array([4, 6, 0, 0, 0, 0, 0, 0]),
       array([ 8,  9, 10,  0,  0,  0,  0,  0]), ...,
       array([ 7, 13, 10,  0,  0,  0,  0,  0]),
       array([7, 0, 0, 0, 0, 0, 0, 0]),
       array([ 7, 10, 16,  0,  0,  0,  0,  0])], dtype=object)

In [180]:
user_item_matrix['genres'].shape

(85083,)

In [154]:
# user_item_matrix['genres'] = user_item_matrix['genres'].values.pad()
# np.array(list(map(np.array, user_item_matrix['genres'].values)))
np.ndarray((user_item_matrix['genres'].shape[0], user_item_matrix['genres'][0].shape[0]), user_item_matrix['genres'])

array([[None, None, None, ..., None, None, None],
       [None, None, None, ..., None, None, None],
       [None, None, None, ..., None, None, None],
       ...,
       [None, None, None, ..., None, None, None],
       [None, None, None, ..., None, None, None],
       [None, None, None, ..., None, None, None]], dtype=object)

In [153]:
for column_name, columnData in user_item_matrix.iteritems():
    print(column_name, columnData.dtypes, columnData.values.shape)

userId int64 (85083,)
movieID int64 (85083,)
rating float64 (85083,)
tags object (85083,)
genres object (85083,)


In [94]:
set([i.shape for i in user_item_matrix['tags']])

{(107,)}

In [123]:
user_item_matrix['genres'].values.reshape()
# user_item_matrix['genres'].shape[0]
# np.ndarray((user_item_matrix['genres'].shape[0], user_item_matrix['genres'][0].shape[0]), user_item_matrix['genres'].values)

TypeError: reshape() takes exactly 1 argument (0 given)

In [71]:
user_item_matrix['genres']

ValueError: setting an array element with a sequence.

In [57]:
user_item_matrix['tags'][0][-10:]

[7705, 10376, 10420, 10422, 11344, 11477, 14562, 15045, 15123, 15170]

,userId,movieID,rating,tags,genres,country
0,1,0,4.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 2, 3, 4]",0
215,1,2,4.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 3, 5]",0
267,1,4,4.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 7, 8, 9]",0
369,1,40,5.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 8, 10, 11, 9]",5
572,1,43,5.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 8, 11, 9]",0


In [ ]:
lb_make = LabelEncoder()
user_item_matrix["country"] = lb_make.fit_transform(user_item_matrix["country"])

# remapping movieIDs so that torch stop yelling at me
new_dict = {a: b for b, a in enumerate(sorted(user_item_matrix['movieID'].unique()))}
user_item_matrix['movieID'] = user_item_matrix['movieID'].map(new_dict)


In [5]:
# user_item_matrix['genres'].sum()
len(set(user_item_matrix['tags'].sum()))

5273

In [40]:
movie_genres = pd.read_csv(hetrec + 'movie_genres.dat', sep='\t')
genre_dict = {v:k for k,v in enumerate(movie_genres['genre'].unique())}
movie_genres['genre'] = movie_genres['genre'].replace(genre_dict)
movie_genres
# to_pad = max(user_item_matrix['genres'].apply(lambda value: len(value)))
# # user_item_matrix['genres'].apply(lambda value: [0]*(to_pad-len(value))+value)
# to_pad

,movieID,country
0,1,USA
1,2,USA
2,3,USA
3,4,USA
4,5,USA


In [19]:
user_item_matrix['country'].nunique()

55

In [26]:
user_item_matrix["genres"].head()

0      [Adventure, Animation, Children, Comedy, Fantasy]
215                                    [Comedy, Romance]
267                            [Action, Crime, Thriller]
369                   [Crime, Horror, Mystery, Thriller]
572                           [Crime, Mystery, Thriller]
Name: genres, dtype: object

In [30]:
genre_dict = {v:k for k,v in enumerate(movie_genres['genre'].unique())}

In [31]:
user_item_matrix["genres"].replace(genre_dict)

0        [Adventure, Animation, Children, Comedy, Fantasy]
215                                      [Comedy, Romance]
267                              [Action, Crime, Thriller]
369                     [Crime, Horror, Mystery, Thriller]
572                             [Crime, Mystery, Thriller]
                               ...                        
41376                        [Action, Adventure, Thriller]
67436                                             [Action]
67443                            [Drama, Sci-Fi, Thriller]
48106                                              [Drama]
67465                               [Drama, Thriller, War]
Name: genres, Length: 85069, dtype: object

In [38]:
lb_make = LabelEncoder()
user_item_matrix["country"] = lb_make.fit_transform(user_item_matrix["country"])


array([51, 51, 51, 29, 51, 51, 51, 51, 51, 51])

In [ ]:
num_user = len(user_item_matrix['userId'].unique()) + 1
num_item = len(user_item_matrix['movieID'].unique()) + 1
num_country = len(user_item_matrix['country'].unique()) + 1
num_genre = len(user_item_matrix['genres'].sum()) + 1
num_tags = len(set(user_item_matrix['tags'].sum())) + 1
print(num_user, num_item, num_country, num_genre, num_tags)


In [223]:
# user_item_matrix.shape
give_test = lambda obj: obj.loc[np.random.choice(obj.index, len(obj.index)//10),:]
test_data = user_item_matrix.groupby('userId', as_index=False).apply(give_test).reset_index(level=0, drop=True)
train_data = user_item_matrix[~user_item_matrix.index.isin(test_data.index)]
train_data.head()

In [4]:
import pandas as pd
import numpy as np

datapath = '../data/'

hetrec = datapath + 'hetrec2011-movielens-2k-v2/'
user_item_matrix = pd.read_csv(datapath + 'ml-latest-small/ratings.csv', usecols=[0, 1, 2]).rename(columns={'movieId':'movieID'})
movies = pd.read_csv(datapath + 'ml-latest-small/movies.csv', usecols=[0, 1])

# # get rt metadata
movies_meta = pd.read_csv(hetrec + 'movies.dat', sep='\t', encoding='raw_unicode_escape').rename(columns={'id': 'movieID'}).drop(['spanishTitle', 'imdbID', 'title', 'imdbPictureURL', 'rtID', 'rtAllCriticsNumReviews', 'rtTopCriticsNumReviews', 'rtPictureURL'], axis=1)
# movies_meta = movies_meta[movies_meta['movieID'].isin(user_item_matrix['movieID'].unique())]

user_item_matrix.head()

# # get the average tags embeddings
# movie_tags = pd.read_csv(hetrec + 'movie_tags.dat', sep='\t')
# tags = pd.read_csv(hetrec + 'tags.dat', sep='\t',  encoding='raw_unicode_escape').rename(columns={'id': 'tagID'})
# movie_tags = pd.merge(movie_tags, tags, on='tagID').sort_values(by=['movieID', 'tagID']).drop(['tagID'], axis=1)
# groupped = movie_tags.groupby('movieID')
# embedded_tags = pd.DataFrame.from_records(movies_meta.apply(get_avg_embedding_for_movie, axis=1), columns=[f'w2v_{i}' for i in range(1, 301)])
# movies_meta = pd.concat([movies_meta, embedded_tags], axis=1)

# user_item_matrix = pd.merge(user_item_matrix, movies_meta, on='movieID').sort_values(by=['userId','movieID'])

# # get a one-hot-encode-esque matrix of genres, then join on them
# movie_genres = pd.read_csv(hetrec + 'movie_genres.dat', sep='\t').pivot_table(index=['movieID'], columns=['genre'], aggfunc=[len], fill_value=0)
# movie_genres.columns = movie_genres.columns.droplevel(0)
# movie_genres = movie_genres.reset_index()
# user_item_matrix = pd.merge(user_item_matrix, movie_genres, on='movieID')

# # get a one-hot-encode matrix of countries, then join on them
# movie_countries = pd.get_dummies(pd.read_csv(hetrec + 'movie_countries.dat', sep='\t'))
# user_item_matrix = pd.merge(user_item_matrix, movie_countries, on='movieID').sort_values(by=['userId','movieID'])
# user_item_matrix.head()

# # for _, columnData in train_data.iteritems()

,userId,movieID,rating,year,rtAllCriticsRating,rtAllCriticsNumFresh,rtAllCriticsNumRotten,rtAllCriticsScore,rtTopCriticsRating,rtTopCriticsNumFresh,rtTopCriticsNumRotten,rtTopCriticsScore,rtAudienceRating,rtAudienceNumRatings,rtAudienceScore
0,1,1,4.0,1995,9,73,0,100,8.5,17,0,100,3.7,102338,81
215,1,3,4.0,1993,5.9,24,12,66,7,5,1,83,3.2,10489,66
267,1,6,4.0,1995,7.7,50,8,86,7.2,14,3,82,3.9,42785,92
369,1,47,5.0,1954,9.2,49,0,100,8.6,10,0,100,4.5,29509,97
572,1,50,5.0,1995,7.5,41,6,87,6.9,12,2,85,4.2,109588,95
